# Counts for Transliterations and Translations

This notebook contains the counting and assessment of transliterations and translations for the ORACC projects.

This will be eventually placed in the Translations Extractions notebook but for easier access can be found here.

# Mount Google Drive folder + imports + steps

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#any necessary imports
import pandas as pd
import zipfile
from zipfile import ZipFile
import json
import requests
from tqdm import tqdm
import os
import errno
import re
import random
import numpy as np
import sys
import copy
import networkx as nx
from pathlib import Path
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

#Set folder for remote drive
#folder = '/content/drive/My Drive/FactGrid Cuneiform (AWCA)/people/Melinee'
folder = '/content/drive/MyDrive/Melinee/'

#importing utils for the method which downloads the current text json files
os.chdir(folder + 'network/utils/')
from utils import oracc_download

# This is a user defined module that searches through the texts to find the entities in the text that
# are people and places, to be imported as nodes into the network
os.chdir(folder + 'network/')
import rank_parser4 as rp

The steps:
1. retrieve the megacatalogue
2. access texts by the form: 'http://oracc.iaas.upenn.edu/[project]/[id_text]/html'
3. analyze the text for transliterations and translations
4. keep a count of the number of transliterations and translations available for ORACC projects

Retrieving the megacatalogue from drive

In [3]:
path = '/content/drive/MyDrive/Melinee/ORACC_DFS/megacatalogue.csv'
megacatalogue = pd.read_csv(path, low_memory=False, index_col=False)

In [4]:
megacatalogue.head(3)

,Unnamed: 0,id_text,langs,project,id_text.1,primary_publication,provenience,pleiades_id,pleiades_coord,excavation_no,...,Q_places,parallels,new_q,subseries,subseries_section,description,oracc_id,sec1,chap,sec2
0,0,P522592,0x08000000,tilbarsip,P522592,Til-Barsip 01,Tell Ahmar (Til Barsip),658410.0,"[38.1191944, 36.6749623]",T 01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,P522593,0x08000000,tilbarsip,P522593,Til-Barsip 02,Tell Ahmar (Til Barsip),658410.0,"[38.1191944, 36.6749623]",T 02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,P522594,0x08000000,tilbarsip,P522594,Til-Barsip 03,Tell Ahmar (Til Barsip),658410.0,"[38.1191944, 36.6749623]",T 03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#These two lines of code are for viewing unique projects and their corresponding texts
#megacatalogue['project'].unique()
#megacatalogue[megacatalogue['project'] == 'tcma/nineveh']['id_text'].unique()

fix_atae = ['tilbarsip','assurmisc', 'stat2', 'stat1', 'wvdog152', 'stat3', 'rfdn17','saab0509', 'tuszhan',
            'szibaniba', 'marqasu', 'guzana','imgurenlil', 'durkatlimmu', 'mallanate', 'huzirina', 'burmarina',
            'ctn2', 'ctn3', 'ctn1',  'ctn6', 'edubba10', 'samal']

fix_saao = ['saa03','saa20', 'saas2', 'saa12', 'saa13', 'saa02', 'saa14', 'saa18', 'saa21', 'saa08', 'saa17', 'saa04',
            'saa10', 'saa15','saa06', 'saa11', 'saa16', 'saa07', 'saa01', 'saa05', 'saa09','saa19']

#add new column 'project_name' that duplicates 'project'
megacatalogue['project_name'] = megacatalogue['project']

#replace all in atae and saao list as main+sub
for i in fix_atae:
  megacatalogue['project_name'] = megacatalogue['project_name'].replace(i, 'atae/'+i)

for i in fix_saao:
  megacatalogue['project_name'] = megacatalogue['project_name'].replace(i, 'saao/'+i)

megacatalogue.shape

(171145, 406)

In [6]:
df = megacatalogue[megacatalogue['project_name'] != 'asbp']
df = df[df['project_name'] != 'armep']['project_name'].unique()
df.sort()
df

array(['adsd/adart1', 'adsd/adart2', 'adsd/adart3', 'adsd/adart5',
       'adsd/adart6', 'aemw/alalakh/idrimi', 'aemw/amarna', 'akklove',
       'ario', 'asbp/ninmed', 'asbp/rlasb', 'atae/assurmisc',
       'atae/burmarina', 'atae/ctn1', 'atae/ctn2', 'atae/ctn3',
       'atae/ctn6', 'atae/durkatlimmu', 'atae/edubba10', 'atae/guzana',
       'atae/huzirina', 'atae/imgurenlil', 'atae/mallanate',
       'atae/marqasu', 'atae/rfdn17', 'atae/saab0509', 'atae/samal',
       'atae/stat1', 'atae/stat2', 'atae/stat3', 'atae/szibaniba',
       'atae/tilbarsip', 'atae/tuszhan', 'atae/wvdog152', 'babcity',
       'blms', 'borsippa', 'btto', 'cams', 'cams/absumu', 'cams/barutu',
       'cams/etana', 'cams/gkab', 'cams/ludlul', 'caspo', 'caspo/akkpm',
       'ccpo', 'cdli', 'ckst', 'cmawro', 'cmawro/cmawr2', 'ctij', 'dcclt',
       'dcclt/ebla', 'dcclt/nineveh', 'dcclt/signlists', 'dccmt', 'dsst',
       'ecut', 'eisl', 'epsd2/admin/ed12', 'epsd2/admin/ed3a',
       'epsd2/admin/ed3b', 'epsd2/admin/

##Automate Extraction of Texts
note: asbp and armep projects have around 20,000 texts each, however through my trials there are 0 transliterations and 0 translations. This forces the program to be strained for 3 more hours.

Note that no transliteration and/or no translation could stem from there not being a supported html "print text" page to scrape. This is the case for the epsd2 files who have transliterations on the oracc project pages.

Estimated run time for first half: 2 hours 11 minutes

Estimated run time for second half: 3 hours 30 minutes

In [7]:
totaltransliterations = 0
totaltranslations = 0

subset = ['adsd/adart1', 'adsd/adart2', 'adsd/adart3', 'adsd/adart5',
       'adsd/adart6', 'aemw/alalakh/idrimi', 'aemw/amarna', 'akklove',
       'ario', 'asbp/ninmed', 'asbp/rlasb', 'atae/assurmisc',
       'atae/burmarina', 'atae/ctn1', 'atae/ctn2', 'atae/ctn3',
       'atae/ctn6', 'atae/durkatlimmu', 'atae/edubba10', 'atae/guzana',
       'atae/huzirina', 'atae/imgurenlil', 'atae/mallanate',
       'atae/marqasu', 'atae/rfdn17', 'atae/saab0509', 'atae/samal',
       'atae/stat1', 'atae/stat2', 'atae/stat3', 'atae/szibaniba',
       'atae/tilbarsip', 'atae/tuszhan', 'atae/wvdog152', 'babcity',
       'blms', 'borsippa', 'btto', 'cams', 'cams/absumu', 'cams/barutu',
       'cams/etana', 'cams/gkab', 'cams/ludlul', 'caspo', 'caspo/akkpm',
       'ccpo', 'cdli', 'ckst', 'cmawro', 'cmawro/cmawr2', 'ctij', 'dcclt',
       'dcclt/ebla', 'dcclt/nineveh', 'dcclt/signlists', 'dccmt', 'dsst',
       'ecut', 'eisl', 'etcsri', 'glass', 'hbtin',
       'lacost', 'nere', 'obel', 'obmc', 'obta', 'qcat']

for j in subset:
  tlitcount = 0
  transcount = 0

  test = megacatalogue[megacatalogue['project_name'] == j].reset_index()

  for i in range(test.shape[0]):
    proj = test['project_name'][i]
    dash_proj = proj.replace("/", "-")
    id_txt = test['id_text'][i]
    #print(dash_proj, id_txt)

    try:
      url = urlopen('https://ccp.yale.edu/' + id_txt)
    except:
      url = urlopen('http://oracc.museum.upenn.edu/' + proj + '/' + id_txt + '/html')
    else:
      url = urlopen('http://oracc.iaas.upenn.edu/' + proj + '/' + id_txt + '/html')

    html = url.read()
    soup = BeautifulSoup(html, 'html.parser')

    transliteration = soup.find_all('p', class_ = 'tt')
    tlittext = ''
    for string in transliteration:
      tlittext += string.get_text()

    translation = soup.find_all('p', class_ = 'tr')
    transtext = ''
    for string in translation:
      transtext += string.get_text()

    #transliteration count
    if len(tlittext) != 0:
      tlitcount += 1
      totaltransliterations += 1

    #translation count
    if len(transtext) != 0:
      transcount += 1
      totaltranslations += 1

  print(dash_proj + ' (' + str(test.shape[0]) + ' texts): transliterations: '+ str(tlitcount) + ', translations: '+ str(transcount))

print(totaltransliterations, totaltranslations)

adsd-adart1 (86 texts): transliterations: 86, translations: 86
adsd-adart2 (177 texts): transliterations: 161, translations: 161
adsd-adart3 (154 texts): transliterations: 154, translations: 154
adsd-adart5 (106 texts): transliterations: 104, translations: 104
adsd-adart6 (180 texts): transliterations: 180, translations: 179
aemw-alalakh-idrimi (1 texts): transliterations: 1, translations: 1
aemw-amarna (379 texts): transliterations: 218, translations: 218
akklove (24 texts): transliterations: 23, translations: 24
ario (175 texts): transliterations: 170, translations: 172
asbp-ninmed (360 texts): transliterations: 76, translations: 75
asbp-rlasb (26 texts): transliterations: 26, translations: 26
atae-assurmisc (7 texts): transliterations: 7, translations: 7
atae-burmarina (44 texts): transliterations: 44, translations: 44
atae-ctn1 (66 texts): transliterations: 33, translations: 33
atae-ctn2 (284 texts): transliterations: 242, translations: 242
atae-ctn3 (133 texts): transliterations: 

In [13]:
subset = ['riao',
       'ribo/babylon10', 'ribo/babylon2', 'ribo/babylon3',
       'ribo/babylon4', 'ribo/babylon5', 'ribo/babylon6', 'ribo/babylon7',
       'ribo/sources', 'rimanum', 'rinap/rinap1', 'rinap/rinap2',
       'rinap/rinap3', 'rinap/rinap4', 'rinap/rinap5', 'rinap/rinap5p1',
       'rinap/sources', 'saao', 'saao/saa01', 'saao/saa02', 'saao/saa03',
       'saao/saa04', 'saao/saa05', 'saao/saa06', 'saao/saa07',
       'saao/saa08', 'saao/saa09', 'saao/saa10', 'saao/saa11',
       'saao/saa12', 'saao/saa13', 'saao/saa14', 'saao/saa15',
       'saao/saa16', 'saao/saa17', 'saao/saa18', 'saao/saa19',
       'saao/saa20', 'saao/saa21', 'saao/saas2', 'suhu', 'tcma/ali1',
       'tcma/amarna', 'tcma/assur', 'tcma/barri', 'tcma/bazmusian',
       'tcma/billa', 'tcma/chuera', 'tcma/emar', 'tcma/fekheriye',
       'tcma/giricano', 'tcma/hana', 'tcma/haradum', 'tcma/hatti',
       'tcma/kartn', 'tcma/kulishinas', 'tcma/laws', 'tcma/miscellaneous',
       'tcma/nineveh', 'tcma/nippur', 'tcma/nuzi', 'tcma/qitar',
       'tcma/rimah', 'tcma/suri', 'tcma/taban', 'tcma/tsa1', 'tcma/tsh1',
       'tcma/ugarit', 'xcat']

for j in subset:
  tlitcount = 0
  transcount = 0

  test = megacatalogue[megacatalogue['project_name'] == j].reset_index()

  for i in range(test.shape[0]):
    proj = test['project_name'][i]
    dash_proj = proj.replace("/", "-")
    id_txt = test['id_text'][i]
    #print(dash_proj, id_txt)

    try:
      url = urlopen('https://ccp.yale.edu/' + id_txt)
    except:
      url = urlopen('http://oracc.museum.upenn.edu/' + proj + '/' + id_txt + '/html')
    else:
      url = urlopen('http://oracc.iaas.upenn.edu/' + proj + '/' + id_txt + '/html')

    html = url.read()
    soup = BeautifulSoup(html, 'html.parser')

    transliteration = soup.find_all('p', class_ = 'tt')
    tlittext = ''
    for string in transliteration:
      tlittext += string.get_text()

    translation = soup.find_all('p', class_ = 'tr')
    transtext = ''
    for string in translation:
      transtext += string.get_text()

    #transliteration count
    if len(tlittext) != 0:
      tlitcount += 1
      totaltransliterations += 1

    #translation count
    if len(transtext) != 0:
      transcount += 1
      totaltranslations += 1

  print(dash_proj + ' (' + str(test.shape[0]) + ' texts): transliterations: '+ str(tlitcount) + ', translations: '+ str(transcount))

print('total transliterations: ' + str(totaltransliterations) + ', total translations: ' + str(totaltranslations))

riao (890 texts): transliterations: 714, translations: 713
ribo-babylon10 (1 texts): transliterations: 1, translations: 1
ribo-babylon2 (38 texts): transliterations: 38, translations: 38
ribo-babylon3 (4 texts): transliterations: 4, translations: 4
ribo-babylon4 (6 texts): transliterations: 6, translations: 6
ribo-babylon5 (1 texts): transliterations: 1, translations: 1
ribo-babylon6 (126 texts): transliterations: 124, translations: 125
ribo-babylon7 (211 texts): transliterations: 192, translations: 192
ribo-sources (396 texts): transliterations: 268, translations: 0
rimanum (345 texts): transliterations: 345, translations: 345
rinap-rinap1 (96 texts): transliterations: 88, translations: 88
rinap-rinap2 (151 texts): transliterations: 134, translations: 145
rinap-rinap3 (259 texts): transliterations: 235, translations: 236
rinap-rinap4 (142 texts): transliterations: 136, translations: 137
rinap-rinap5 (302 texts): transliterations: 300, translations: 300
rinap-rinap5p1 (1 texts): transl

In [14]:
print('total transliterations: ' + str(totaltransliterations) + ', total translations: ' + str(totaltranslations))

total transliterations: 22256, total translations: 18448


##EXAMPLE: Get the Transliterations and English Translation of a Single Text

In this section, there
will be a count for if a transliteration and english translation are found for a given url.

In [ ]:
#changes directory to English Translations folder
os.chdir(folder +'ORACC_TRANSLATIONS/ENG_TRANSLATIONS')

#given a url, parse through and create a soup of html text
url = urlopen('http://oracc.iaas.upenn.edu/adsd/adart5/X500001/html')
html = url.read()
soup = BeautifulSoup(html, 'html.parser')

#for english translation:
#create a variable 'soupy' that contains all elements of the p class translations
#using a for loop, get all texts. *get_text() only works for single element
soupy = soup.find_all()
soupy = soup.find_all('p', class_ = 'tr')
text = ''
for string in soupy:
  text += string.get_text()

text

#for transliteration:
soupy2 = soup.find_all()
soupy2 = soup.find_all('p', class_ = 'tt')
text2 = ''
for string in soupy2:
  text2 += string.get_text()

text2

'ina a-mat dEN u dGAŠAN-ia₂ liš-lim1.40 MU SAG NAM.LUGAL.LA [x x x]ŠE 5 ITU 14 U₄.ZAL GAR ad₂ ⸢x⸣ [x x]2.10 MU 1 KIN ⸢15?⸣ GAR ina SI SAR [x (x)][x] ULU₃ ⸢GIN⸣ ad₂ ŠU₂ KIN ⸢DIR⸣[ZIZ₂] ⸢14⸣ GAR 1.40 ana ZALAG₂ ⸢TAG₄?⸣[MU 2] ⸢NE⸣ 14 TIL GAR[ZIZ₂] DIB[MU 3 NE] ⸢DIB⸣ KIN DIR[x x x x] TIL ⸢x⸣[...] ⸢x⸣ [...]'

#Total counts:

In the form project name (total number of texts in the megacatalogue): number of transliterations and the number of translations found

**note that this number of transliterations and translations is based on if a given text has an available html to parse through. (Via the 'print text' option on the oracc projects website


**total transliterations: 22256, total translations: 18448**

adsd-adart1 (86 texts): transliterations: 86, translations: 86

adsd-adart2 (177 texts): transliterations: 161, translations: 161

adsd-adart3 (154 texts): transliterations: 154, translations: 154

adsd-adart5 (106 texts): transliterations: 104, translations: 104

adsd-adart6 (180 texts): transliterations: 180, translations: 179

aemw-alalakh-idrimi (1 texts): transliterations: 1, translations: 1

aemw-amarna (379 texts): transliterations: 218, translations: 218

akklove (24 texts): transliterations: 23, translations: 24

ario (175 texts): transliterations: 170, translations: 172

asbp-ninmed (360 texts): transliterations: 76, translations: 75

asbp-rlasb (26 texts): transliterations: 26, translations: 26

atae-assurmisc (7 texts): transliterations: 7, translations: 7

atae-burmarina (44 texts): transliterations: 44, translations: 44

atae-ctn1 (66 texts): transliterations: 33, translations: 33

atae-ctn2 (284 texts): transliterations: 242, translations: 242

atae-ctn3 (133 texts): transliterations: 114, translations: 118

atae-ctn6 (120 texts): transliterations: 117, translations: 117

atae-durkatlimmu (210 texts): transliterations: 210, translations: 1

atae-edubba10 (55 texts): transliterations: 55, translations: 55

atae-guzana (226 texts): transliterations: 207, translations: 103

atae-huzirina (19 texts): transliterations: 18, translations: 17

atae-imgurenlil (30 texts): transliterations: 30, translations: 30

atae-mallanate (74 texts): transliterations: 73, translations: 48

atae-marqasu (46 texts): transliterations: 44, translations: 44

atae-rfdn17 (34 texts): transliterations: 34, translations: 34

atae-saab0509 (144 texts): transliterations: 142, translations: 91

atae-samal (2 texts): transliterations: 2, translations: 2

atae-stat1 (52 texts): transliterations: 51, translations: 52

atae-stat2 (328 texts): transliterations: 328, translations: 328

atae-stat3 (113 texts): transliterations: 110, translations: 110

atae-szibaniba (22 texts): transliterations: 22, translations: 1

atae-tilbarsip (22 texts): transliterations: 20, translations: 20

atae-tuszhan (35 texts): transliterations: 33, translations: 33

atae-wvdog152 (88 texts): transliterations: 85, translations: 85

babcity (276 texts): transliterations: 219, translations: 218

blms (382 texts): transliterations: 371, translations: 355

borsippa (224 texts): transliterations: 224, translations: 224

btto (25 texts): transliterations: 6, translations: 1

cams (12 texts): transliterations: 0, translations: 0

cams-absumu (20 texts): transliterations: 0, translations: 0

cams-barutu (2 texts): transliterations: 2, translations: 2

cams-etana (14 texts): transliterations: 14, translations: 0

cams-gkab (574 texts): transliterations: 534, translations: 567

cams-ludlul (14 texts): transliterations: 13, translations: 0

caspo (252 texts): transliterations: 250, translations: 250

caspo-akkpm (4 texts): transliterations: 4, translations: 4

ccpo (183 texts): transliterations: 156, translations: 180

cdli (19 texts): transliterations: 0, translations: 0

ckst (90 texts): transliterations: 85, translations: 71

cmawro (337 texts): transliterations: 0, translations: 0

cmawro-cmawr2 (1 texts): transliterations: 0, translations: 0

ctij (134 texts): transliterations: 132, translations: 60

dcclt (5481 texts): transliterations: 3562, translations: 1237

dcclt-ebla (137 texts): transliterations: 129, translations: 99

dcclt-nineveh (656 texts): transliterations: 78, translations: 438

dcclt-signlists (166 texts): transliterations: 32, translations: 84

dccmt (1141 texts): transliterations: 149, translations: 115

dsst (124 texts): transliterations: 56, translations: 0

ecut (823 texts): transliterations: 730, translations: 651

eisl (341 texts): transliterations: 57, translations: 11

etcsri (1456 texts): transliterations: 0, translations: 1445

glass (15 texts): transliterations: 15, translations: 15

hbtin (522 texts): transliterations: 477, translations: 299

lacost (5 texts): transliterations: 2, translations: 1

nere (1 texts): transliterations: 1, translations: 1

obel (409 texts): transliterations: 185, translations: 174

obmc (175 texts): transliterations: 158, translations: 155

obta (59 texts): transliterations: 0, translations: 0

qcat (648 texts): transliterations: 0, translations: 0

riao (890 texts): transliterations: 714, translations: 713

ribo-babylon10 (1 texts): transliterations: 1, translations: 1

ribo-babylon2 (38 texts): transliterations: 38, translations: 38

ribo-babylon3 (4 texts): transliterations: 4, translations: 4

ribo-babylon4 (6 texts): transliterations: 6, translations: 6

ribo-babylon5 (1 texts): transliterations: 1, translations: 1

ribo-babylon6 (126 texts): transliterations: 124, translations: 125

ribo-babylon7 (211 texts): transliterations: 192, translations: 192

ribo-sources (396 texts): transliterations: 268, translations: 0

rimanum (345 texts): transliterations: 345, translations: 345

rinap-rinap1 (96 texts): transliterations: 88, translations: 88

rinap-rinap2 (151 texts): transliterations: 134, translations: 145

rinap-rinap3 (259 texts): transliterations: 235, translations: 236

rinap-rinap4 (142 texts): transliterations: 136, translations: 137

rinap-rinap5 (302 texts): transliterations: 300, translations: 300

rinap-rinap5p1 (1 texts): transliterations: 0, translations: 0

rinap-sources (2199 texts): transliterations: 1967, translations: 0

saao (1 texts): transliterations: 0, translations: 0

saao-saa01 (261 texts): transliterations: 260, translations: 260

saao-saa02 (15 texts): transliterations: 15, translations: 15

saao-saa03 (49 texts): transliterations: 47, translations: 47

saao-saa04 (361 texts): transliterations: 348, translations: 348

saao-saa05 (297 texts): transliterations: 295, translations: 295

saao-saa06 (350 texts): transliterations: 350, translations: 350

saao-saa07 (219 texts): transliterations: 213, translations: 218

saao-saa08 (567 texts): transliterations: 566, translations: 566

saao-saa09 (11 texts): transliterations: 11, translations: 11

saao-saa10 (389 texts): transliterations: 389, translations: 389

saao-saa11 (234 texts): transliterations: 230, translations: 234

saao-saa12 (92 texts): transliterations: 91, translations: 92

saao-saa13 (210 texts): transliterations: 210, translations: 210

saao-saa14 (479 texts): transliterations: 479, translations: 479

saao-saa15 (391 texts): transliterations: 389, translations: 389

saao-saa16 (246 texts): transliterations: 246, translations: 246

saao-saa17 (207 texts): transliterations: 207, translations: 207

saao-saa18 (204 texts): transliterations: 203, translations: 203

saao-saa19 (229 texts): transliterations: 229, translations: 229

saao-saa20 (51 texts): transliterations: 51, translations: 51

saao-saa21 (161 texts): transliterations: 161, translations: 161

saao-saas2 (21 texts): transliterations: 10, translations: 2

suhu (33 texts): transliterations: 26, translations: 27

tcma-ali1 (11 texts): transliterations: 11, translations: 11

tcma-amarna (1 texts): transliterations: 1, translations: 1

tcma-assur (1114 texts): transliterations: 1043, translations: 975

tcma-barri (6 texts): transliterations: 3, translations: 3

tcma-bazmusian (7 texts): transliterations: 7, translations: 4

tcma-billa (66 texts): transliterations: 65, translations: 57

tcma-chuera (97 texts): transliterations: 92, translations: 78

tcma-emar (3 texts): transliterations: 2, translations: 2

tcma-fekheriye (15 texts): transliterations: 14, translations: 12

tcma-giricano (15 texts): transliterations: 15, translations: 15

tcma-hana (2 texts): transliterations: 2, translations: 2

tcma-haradum (4 texts): transliterations: 2, translations: 2

tcma-hatti (10 texts): transliterations: 10, translations: 7

tcma-kartn (62 texts): transliterations: 57, translations: 58

tcma-kulishinas (13 texts): transliterations: 10, translations: 9

tcma-laws (45 texts): transliterations: 45, translations: 41

tcma-miscellaneous (29 texts): transliterations: 24, translations: 24

tcma-nineveh (1 texts): transliterations: 1, translations: 0

tcma-nippur (4 texts): transliterations: 4, translations: 4

tcma-nuzi (1 texts): transliterations: 1, translations: 1

tcma-qitar (1 texts): transliterations: 1, translations: 1

tcma-rimah (123 texts): transliterations: 123, translations: 98

tcma-suri (2 texts): transliterations: 0, translations: 0

tcma-taban (23 texts): transliterations: 13, translations: 12

tcma-tsa1 (28 texts): transliterations: 17, translations: 12

tcma-tsh1 (334 texts): transliterations: 248, translations: 183

tcma-ugarit (5 texts): transliterations: 5, translations: 4

xcat (12 texts): transliterations: 0, translations: 0